In [9]:
# Sauce: https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py

In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [5]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [8]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

95 [D loss: 0.016368, acc.: 100.00%] [G loss: 4.708967]
96 [D loss: 0.015619, acc.: 100.00%] [G loss: 4.827609]
97 [D loss: 0.013036, acc.: 100.00%] [G loss: 4.885368]
98 [D loss: 0.010378, acc.: 100.00%] [G loss: 4.913558]
99 [D loss: 0.011721, acc.: 100.00%] [G loss: 4.873065]
100 [D loss: 0.010766, acc.: 100.00%] [G loss: 4.840436]
101 [D loss: 0.013890, acc.: 100.00%] [G loss: 4.929687]
102 [D loss: 0.014116, acc.: 100.00%] [G loss: 4.646921]
103 [D loss: 0.022983, acc.: 100.00%] [G loss: 4.944677]
104 [D loss: 0.017314, acc.: 100.00%] [G loss: 4.720614]
105 [D loss: 0.010187, acc.: 100.00%] [G loss: 4.712412]
106 [D loss: 0.013718, acc.: 100.00%] [G loss: 4.587002]
107 [D loss: 0.023308, acc.: 100.00%] [G loss: 5.118883]
108 [D loss: 0.035437, acc.: 100.00%] [G loss: 4.796826]
109 [D loss: 0.016254, acc.: 100.00%] [G loss: 4.809444]
110 [D loss: 0.010962, acc.: 100.00%] [G loss: 5.034156]
111 [D loss: 0.019901, acc.: 100.00%] [G loss: 4.854660]
112 [D loss: 0.038743, acc.: 98.44%]

241 [D loss: 0.386458, acc.: 79.69%] [G loss: 2.950575]
242 [D loss: 0.718042, acc.: 54.69%] [G loss: 2.227224]
243 [D loss: 0.412470, acc.: 76.56%] [G loss: 3.044823]
244 [D loss: 0.571413, acc.: 71.88%] [G loss: 1.955911]
245 [D loss: 0.361857, acc.: 84.38%] [G loss: 3.090756]
246 [D loss: 0.956986, acc.: 37.50%] [G loss: 1.096668]
247 [D loss: 0.348683, acc.: 84.38%] [G loss: 2.479473]
248 [D loss: 0.777132, acc.: 57.81%] [G loss: 1.417459]
249 [D loss: 0.438142, acc.: 75.00%] [G loss: 2.520925]
250 [D loss: 1.091479, acc.: 31.25%] [G loss: 0.766620]
251 [D loss: 0.481574, acc.: 64.06%] [G loss: 1.944222]
252 [D loss: 0.595721, acc.: 64.06%] [G loss: 1.487019]
253 [D loss: 0.572166, acc.: 67.19%] [G loss: 1.986136]
254 [D loss: 0.676114, acc.: 59.38%] [G loss: 1.567997]
255 [D loss: 0.509393, acc.: 65.62%] [G loss: 2.126276]
256 [D loss: 1.023786, acc.: 26.56%] [G loss: 0.840516]
257 [D loss: 0.587718, acc.: 60.94%] [G loss: 1.434669]
258 [D loss: 0.621950, acc.: 57.81%] [G loss: 1.

388 [D loss: 0.656212, acc.: 53.12%] [G loss: 0.685604]
389 [D loss: 0.663913, acc.: 51.56%] [G loss: 0.695577]
390 [D loss: 0.665538, acc.: 54.69%] [G loss: 0.701885]
391 [D loss: 0.651540, acc.: 50.00%] [G loss: 0.719950]
392 [D loss: 0.673381, acc.: 53.12%] [G loss: 0.705053]
393 [D loss: 0.664097, acc.: 48.44%] [G loss: 0.699805]
394 [D loss: 0.646321, acc.: 54.69%] [G loss: 0.701442]
395 [D loss: 0.671436, acc.: 53.12%] [G loss: 0.702178]
396 [D loss: 0.666739, acc.: 50.00%] [G loss: 0.686547]
397 [D loss: 0.674241, acc.: 50.00%] [G loss: 0.685652]
398 [D loss: 0.668970, acc.: 53.12%] [G loss: 0.690427]
399 [D loss: 0.675670, acc.: 53.12%] [G loss: 0.714886]
400 [D loss: 0.664307, acc.: 54.69%] [G loss: 0.696582]
401 [D loss: 0.668294, acc.: 50.00%] [G loss: 0.720956]
402 [D loss: 0.684833, acc.: 50.00%] [G loss: 0.703986]
403 [D loss: 0.669860, acc.: 50.00%] [G loss: 0.682774]
404 [D loss: 0.687342, acc.: 48.44%] [G loss: 0.679501]
405 [D loss: 0.658129, acc.: 50.00%] [G loss: 0.

535 [D loss: 0.608717, acc.: 67.19%] [G loss: 0.782325]
536 [D loss: 0.647942, acc.: 56.25%] [G loss: 0.819072]
537 [D loss: 0.664121, acc.: 46.88%] [G loss: 0.761966]
538 [D loss: 0.652591, acc.: 57.81%] [G loss: 0.767167]
539 [D loss: 0.622838, acc.: 60.94%] [G loss: 0.803214]
540 [D loss: 0.629508, acc.: 62.50%] [G loss: 0.814496]
541 [D loss: 0.625621, acc.: 68.75%] [G loss: 0.823921]
542 [D loss: 0.603137, acc.: 78.12%] [G loss: 0.786882]
543 [D loss: 0.606759, acc.: 78.12%] [G loss: 0.812496]
544 [D loss: 0.618196, acc.: 67.19%] [G loss: 0.800160]
545 [D loss: 0.640034, acc.: 62.50%] [G loss: 0.810740]
546 [D loss: 0.645239, acc.: 56.25%] [G loss: 0.788779]
547 [D loss: 0.633000, acc.: 64.06%] [G loss: 0.804927]
548 [D loss: 0.659121, acc.: 57.81%] [G loss: 0.770001]
549 [D loss: 0.610873, acc.: 59.38%] [G loss: 0.755998]
550 [D loss: 0.643068, acc.: 57.81%] [G loss: 0.779339]
551 [D loss: 0.618105, acc.: 65.62%] [G loss: 0.795504]
552 [D loss: 0.591523, acc.: 78.12%] [G loss: 0.

...

29342 [D loss: 0.659288, acc.: 60.94%] [G loss: 0.954338]
29343 [D loss: 0.697577, acc.: 57.81%] [G loss: 0.987849]
29344 [D loss: 0.614360, acc.: 64.06%] [G loss: 1.047775]
29345 [D loss: 0.608609, acc.: 68.75%] [G loss: 1.054919]
29346 [D loss: 0.623794, acc.: 67.19%] [G loss: 1.109993]
29347 [D loss: 0.678386, acc.: 53.12%] [G loss: 1.074563]
29348 [D loss: 0.636907, acc.: 64.06%] [G loss: 1.093167]
29349 [D loss: 0.619270, acc.: 68.75%] [G loss: 0.986091]
29350 [D loss: 0.591111, acc.: 64.06%] [G loss: 0.906811]
29351 [D loss: 0.717729, acc.: 59.38%] [G loss: 0.951490]
29352 [D loss: 0.629900, acc.: 62.50%] [G loss: 0.925716]
29353 [D loss: 0.632838, acc.: 65.62%] [G loss: 1.075136]
29354 [D loss: 0.674946, acc.: 64.06%] [G loss: 1.020857]
29355 [D loss: 0.677710, acc.: 67.19%] [G loss: 0.950650]
29356 [D loss: 0.538366, acc.: 73.44%] [G loss: 1.150374]
29357 [D loss: 0.601460, acc.: 75.00%] [G loss: 1.000353]
29358 [D loss: 0.673271, acc.: 54.69%] [G loss: 0.964288]
29359 [D loss:

29485 [D loss: 0.706580, acc.: 59.38%] [G loss: 1.019961]
29486 [D loss: 0.729508, acc.: 64.06%] [G loss: 1.022010]
29487 [D loss: 0.677642, acc.: 53.12%] [G loss: 1.134500]
29488 [D loss: 0.699217, acc.: 60.94%] [G loss: 0.977663]
29489 [D loss: 0.588942, acc.: 78.12%] [G loss: 1.121699]
29490 [D loss: 0.724044, acc.: 56.25%] [G loss: 1.000203]
29491 [D loss: 0.592836, acc.: 71.88%] [G loss: 0.932461]
29492 [D loss: 0.583883, acc.: 70.31%] [G loss: 0.924689]
29493 [D loss: 0.616972, acc.: 71.88%] [G loss: 1.101530]
29494 [D loss: 0.638984, acc.: 64.06%] [G loss: 1.009566]
29495 [D loss: 0.632332, acc.: 62.50%] [G loss: 0.880406]
29496 [D loss: 0.645407, acc.: 65.62%] [G loss: 0.964168]
29497 [D loss: 0.592171, acc.: 71.88%] [G loss: 0.903250]
29498 [D loss: 0.677722, acc.: 62.50%] [G loss: 1.197822]
29499 [D loss: 0.633380, acc.: 62.50%] [G loss: 1.014298]
29500 [D loss: 0.676006, acc.: 59.38%] [G loss: 1.107599]
29501 [D loss: 0.575748, acc.: 71.88%] [G loss: 1.106434]
29502 [D loss:

29628 [D loss: 0.639944, acc.: 64.06%] [G loss: 0.994263]
29629 [D loss: 0.705078, acc.: 56.25%] [G loss: 1.036820]
29630 [D loss: 0.744621, acc.: 43.75%] [G loss: 1.105730]
29631 [D loss: 0.631181, acc.: 57.81%] [G loss: 0.946776]
29632 [D loss: 0.668737, acc.: 54.69%] [G loss: 0.986548]
29633 [D loss: 0.592727, acc.: 65.62%] [G loss: 1.036565]
29634 [D loss: 0.599642, acc.: 70.31%] [G loss: 1.117324]
29635 [D loss: 0.638280, acc.: 64.06%] [G loss: 1.013095]
29636 [D loss: 0.668534, acc.: 60.94%] [G loss: 1.025771]
29637 [D loss: 0.628501, acc.: 65.62%] [G loss: 0.992569]
29638 [D loss: 0.580649, acc.: 68.75%] [G loss: 1.062956]
29639 [D loss: 0.693457, acc.: 53.12%] [G loss: 0.973232]
29640 [D loss: 0.640680, acc.: 62.50%] [G loss: 1.037845]
29641 [D loss: 0.769867, acc.: 43.75%] [G loss: 0.930405]
29642 [D loss: 0.628200, acc.: 59.38%] [G loss: 0.988509]
29643 [D loss: 0.635126, acc.: 67.19%] [G loss: 1.108173]
29644 [D loss: 0.563326, acc.: 75.00%] [G loss: 1.193760]
29645 [D loss:

29770 [D loss: 0.620037, acc.: 62.50%] [G loss: 1.029225]
29771 [D loss: 0.632800, acc.: 65.62%] [G loss: 0.908973]
29772 [D loss: 0.645701, acc.: 59.38%] [G loss: 1.101917]
29773 [D loss: 0.627603, acc.: 65.62%] [G loss: 1.021081]
29774 [D loss: 0.549719, acc.: 76.56%] [G loss: 1.006464]
29775 [D loss: 0.625228, acc.: 67.19%] [G loss: 0.982821]
29776 [D loss: 0.620621, acc.: 70.31%] [G loss: 0.977222]
29777 [D loss: 0.638477, acc.: 71.88%] [G loss: 1.037217]
29778 [D loss: 0.681019, acc.: 56.25%] [G loss: 0.955686]
29779 [D loss: 0.643980, acc.: 60.94%] [G loss: 1.014717]
29780 [D loss: 0.582971, acc.: 71.88%] [G loss: 1.002825]
29781 [D loss: 0.611284, acc.: 62.50%] [G loss: 0.989136]
29782 [D loss: 0.544120, acc.: 75.00%] [G loss: 1.022918]
29783 [D loss: 0.717053, acc.: 56.25%] [G loss: 1.155675]
29784 [D loss: 0.659960, acc.: 60.94%] [G loss: 1.044620]
29785 [D loss: 0.577778, acc.: 71.88%] [G loss: 1.017063]
29786 [D loss: 0.680703, acc.: 51.56%] [G loss: 0.966207]
29787 [D loss:

29913 [D loss: 0.791533, acc.: 43.75%] [G loss: 0.958471]
29914 [D loss: 0.626434, acc.: 62.50%] [G loss: 0.992750]
29915 [D loss: 0.553735, acc.: 73.44%] [G loss: 1.130485]
29916 [D loss: 0.659180, acc.: 64.06%] [G loss: 0.956130]
29917 [D loss: 0.618699, acc.: 65.62%] [G loss: 1.077100]
29918 [D loss: 0.610335, acc.: 62.50%] [G loss: 0.914061]
29919 [D loss: 0.620276, acc.: 65.62%] [G loss: 0.918581]
29920 [D loss: 0.635873, acc.: 64.06%] [G loss: 1.091072]
29921 [D loss: 0.599992, acc.: 70.31%] [G loss: 1.092416]
29922 [D loss: 0.668050, acc.: 65.62%] [G loss: 0.968657]
29923 [D loss: 0.600262, acc.: 64.06%] [G loss: 0.894330]
29924 [D loss: 0.675329, acc.: 62.50%] [G loss: 1.015018]
29925 [D loss: 0.642876, acc.: 62.50%] [G loss: 1.022472]
29926 [D loss: 0.601411, acc.: 67.19%] [G loss: 1.176315]
29927 [D loss: 0.609031, acc.: 65.62%] [G loss: 0.970411]
29928 [D loss: 0.705166, acc.: 53.12%] [G loss: 1.183687]
29929 [D loss: 0.662383, acc.: 64.06%] [G loss: 0.963092]
29930 [D loss: